<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Parsing Output

Let's set up a Chat Model:

In [ ]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate,ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
api_key = open("C://Users//Marcial//Desktop//desktop_openai.txt").read()
model = ChatOpenAI(openai_api_key=api_key)

## List Parsing 

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
output_parser = CommaSeparatedListOutputParser()

In [ ]:
format_instructions = output_parser.get_format_instructions()

In [ ]:
print(format_instructions)

In [ ]:
reply = "one, two, three"
output_parser.parse("one, two, three")

In [ ]:
human_template = '{request} {format_instructions}'
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions())

In [ ]:
request = chat_prompt.format_prompt(request="give me 5 characteristics of dogs",
                   format_instructions = output_parser.get_format_instructions()).to_messages()

In [ ]:
result = model(request)

In [ ]:
result.content

In [ ]:
# Convert to desired output:
output_parser.parse(result.content)

## Datetime Parser 

In [ ]:
from langchain.output_parsers import DatetimeOutputParser

In [ ]:
output_parser = DatetimeOutputParser()

In [ ]:
print(output_parser.get_format_instructions())

In [ ]:
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

In [ ]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

In [ ]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [ ]:
result = model(request,temperature=0)

In [ ]:
# Careful with this, it sometimes will include extra information!
result.content

In [ ]:
result.content

In [ ]:
output_parser.parse(result.content)

---

# Methods to Fix Parsing Issues

## Auto-Fix Parser

In [ ]:
from langchain.output_parsers import OutputFixingParser

output_parser = DatetimeOutputParser()

misformatted = result.content

In [ ]:
misformatted

In [ ]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=model)

In [ ]:
new_parser.parse(misformatted)

____
### Fixing via System Prompt:

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template("You always reply to questions only in datetime patterns.")
template_text = "{request}\n{format_instructions}"
human_prompt=HumanMessagePromptTemplate.from_template(template_text)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_prompt,human_prompt])

In [ ]:
print(chat_prompt.format(request="When was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ))

In [ ]:
request = chat_prompt.format_prompt(request="What date was the 13th Amendment ratified in the US?",
                   format_instructions=output_parser.get_format_instructions()
                   ).to_messages()

In [ ]:
result = model(request,temperature=0)

In [ ]:
result.content

In [ ]:
output_parser.parse(result.content)

Be careful! This could technically be construed as wrong? The full details from Wikipedia:

    27th state to ratify was Georgia: December 6, 1865

    Having been ratified by the legislatures of three-fourths of the states (27 of the 36 states, including those that had been in rebellion), Secretary of State Seward, on December 18, 1865, certified that the Thirteenth Amendment had become valid, to all intents and purposes, as a part of the Constitution.

You also have the issue of states leaving the union, which complicates what a full ratification means at that time. It kind of depends what is meant by the word "ratified"!

## Pydantic JSON Parser
You should also be aware of OpenAI's own JSON offerings (which are still quite new at this time!): https://platform.openai.com/docs/guides/gpt/function-calling


In [ ]:
#pip install pydantic

In [ ]:
from langchain.output_parsers import PydanticOutputParser

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class Scientist(BaseModel):
    
    name: str = Field(description="Name of a Scientist")
    discoveries: list = Field(description="Python list of discoveries")

In [ ]:
query = 'Name a famous scientist and a list of their discoveries' 

In [ ]:
parser = PydanticOutputParser(pydantic_object=Scientist)

In [ ]:
print(parser.get_format_instructions())

In [ ]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query="Tell me about a famous scientist")

output = model(_input.to_string())

parser.parse(output)